In [13]:
#Simple Airline booking and information providing chatbot
#with interactive interface and creating and using tools with openai

In [14]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [15]:
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [27]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [28]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [29]:
available_flights={"london": "9", "paris": "8", "tokyo": "10", "berlin": "4"}
def get_flights_available(destination_city):
    print(f"Tool get_flights_available called for {destination_city}")
    city=destination_city.lower()
    return available_flights.get(city,"Unknown")

In [30]:
def booking(name, destination_city, seatnum):
    print(f"Tool booking called for {name}, {destination_city}, {seatnum}")
    city = destination_city.lower()
    booking_data = [{"name": name, "destination": city, "number of seats": seatnum}]
    with open("bookings.txt", 'a') as f:
        json.dump(booking_data, f)  # Write as JSON
        f.write("\n")  # New line for readability
    return f"Booking Complete with this information: {booking_data}"

In [31]:
booking_func={
    "name":"booking",
    "description":"Whenever the user requests to make a booking, we will use this tool to append the booking details into a text file.",
    "parameters":{
        "type":"object",
        "properties":{
        "destination_city":{
            "type":"string",
            "description":"the final city the customer wants to travel to",},
        "name":{
            "type":"string",
            "description":"Customer's name"},
        "seatnum":{
            "type":"string",
            "description":"The number of flight seats the customer wants to book or reserve"},
    },
    "required":["destination_city","name","seatnum"],
    "additionalProperties":False
}}


In [32]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [33]:
availability={
    "name":"get_flights_available",
    "description":"Gets the value of the number of flights available to the destination city in the current week.\
    Call this whenever you need to know the number of available fligts that week for example,\
    when the customer asks 'How many flights are available to the city?'",
    "parameters":{
        "type":"object",
        "properties":{
            "destination_city":{
                "type":"string",
                "description":"The city that the customer wants to know number of flights available to",
            },
        },
        "required":["destination_city"],
        "additionalProperties":False
    }
}

In [34]:
tools = [{"type": "function", "function": price_function},{"type":"function","function":availability},{"type":"function","function":booking_func}]

In [35]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content



In [36]:
def handle_tool_call(message):
    responses = []  
    cities = []     

    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        name=arguments.get('name')
        seatnum=arguments.get('seatnum')

        if tool_name == "get_ticket_price":
            price = get_ticket_price(city)
            response = {
                "role": "tool",
                "content": json.dumps({"destination_city": city, "price": price}),
                "tool_call_id": tool_call.id
            }
        elif tool_name == "get_flights_available":
            availability = get_flights_available(city)
            response = {
                "role": "tool","content": json.dumps({"destination_city": city, "availability": availability}),
                "tool_call_id": tool_call.id
            }
        elif tool_name=="booking":
            status=booking(name,city,seatnum)
            response={
                "role":"tool",
                "content":json.dumps({"status":status}),
                "tool_call_id":tool_call.id
            }
        else:
            response = {
                "role": "tool",
                "content": json.dumps({"error": f"Unknown tool: {tool_name}"}),
                "tool_call_id": tool_call.id
            }

        responses.append(response)
        cities.append(city) 

    return responses, cities

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()